In [1]:
# Create a spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", "/user/{username}/warehouse"). \
config("spark.dynamicAllocation.enabled", "false"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
# Define the structure for reading the JSON files under the specified location

user_sms_schema = StructType([
    StructField("user_id", IntegerType(), nullable=False),
    StructField("user_first_name", StringType(), nullable=False),
    StructField("user_last_name", StringType(), nullable=False),
    StructField("user_email", StringType(), nullable=False),
    StructField("user_gender", StringType(), nullable=False),
    StructField("user_phone_numbers", ArrayType(StringType()), nullable=False),
    StructField("user_address", StructType([
        StructField("street", StringType(), nullable=False),
        StructField("city", StringType(), nullable=False),
        StructField("state", StringType(), nullable=False),
        StructField("postal_code", StringType(), nullable=False), ]) , nullable=False)])

In [4]:
# Reading the json files in the specified HDFS location and storing it in a dataframe

user_sms_df = spark.read.format("json").schema(user_sms_schema).load("/public/sms/users")

In [5]:
# To get a high level view of what the data looks like

user_sms_df.show(truncate=False)

+-------+---------------+--------------+--------------------------------+-----------+------------------------------------------------------------+----------------------------------------------------------+
|user_id|user_first_name|user_last_name|user_email                      |user_gender|user_phone_numbers                                          |user_address                                              |
+-------+---------------+--------------+--------------------------------+-----------+------------------------------------------------------------+----------------------------------------------------------+
|200001 |Eirena         |Cutsforth     |ecutsforth0@wisc.edu            |Female     |[4197404036, 9173828061, 4027467939, 3371963735]            |{8 Warrior Drive, Dallas, Texas, 75358}                   |
|200002 |Marja          |Shopcott      |mshopcott1@hexun.com            |Female     |[9542037028, 2128289053, 4068088779, 2621495368, 9038414778]|{66 Prairieview Terrace, Jolie

In [6]:
# Slicing of street, city, state and post code from user_address(stored in nested JSON) along with count of phone numbers and creating new columns

user_sms_df = user_sms_df.withColumn("user_street", col("user_address.street")) \
.withColumn("user_city", col("user_address.city")) \
.withColumn("user_state", col("user_address.state")) \
.withColumn("user_postal_code", col("user_address.postal_code")) \
.withColumn("countOf_PhoneNumbers", size(col("user_phone_numbers")))

In [7]:
user_sms_df.show()

+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------------+----------+----------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|user_gender|  user_phone_numbers|        user_address|         user_street|        user_city|user_state|user_postal_code|countOf_PhoneNumbers|
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------------+----------+----------------+--------------------+
| 200001|         Eirena|     Cutsforth|ecutsforth0@wisc.edu|     Female|[4197404036, 9173...|{8 Warrior Drive,...|     8 Warrior Drive|           Dallas|     Texas|           75358|                   4|
| 200002|          Marja|      Shopcott|mshopcott1@hexun.com|     Female|[9542037028, 2128...|{66 Prairieview T...|66 Prairieview Te...|           Joliet|  Illinois|           60435|  

In [8]:
# Converting the df to a managed spark table so we can work of SQL format querying style

user_sms_df.createOrReplaceTempView("user_sms_data")
spark.sql("describe extended user_sms_data").collect()

[Row(col_name='user_id', data_type='int', comment=None),
 Row(col_name='user_first_name', data_type='string', comment=None),
 Row(col_name='user_last_name', data_type='string', comment=None),
 Row(col_name='user_email', data_type='string', comment=None),
 Row(col_name='user_gender', data_type='string', comment=None),
 Row(col_name='user_phone_numbers', data_type='array<string>', comment=None),
 Row(col_name='user_address', data_type='struct<street:string,city:string,state:string,postal_code:string>', comment=None),
 Row(col_name='user_street', data_type='string', comment=None),
 Row(col_name='user_city', data_type='string', comment=None),
 Row(col_name='user_state', data_type='string', comment=None),
 Row(col_name='user_postal_code', data_type='string', comment=None),
 Row(col_name='countOf_PhoneNumbers', data_type='int', comment=None)]

In [9]:
spark.sql("select * from user_sms_data")

user_id,user_first_name,user_last_name,user_email,user_gender,user_phone_numbers,user_address,user_street,user_city,user_state,user_postal_code,countOf_PhoneNumbers
200001,Eirena,Cutsforth,ecutsforth0@wisc.edu,Female,"[4197404036, 9173...","{8 Warrior Drive,...",8 Warrior Drive,Dallas,Texas,75358,4
200002,Marja,Shopcott,mshopcott1@hexun.com,Female,"[9542037028, 2128...",{66 Prairieview T...,66 Prairieview Te...,Joliet,Illinois,60435,5
200003,Dawn,Tointon,dtointon2@ucsd.edu,Female,"[9523035647, 2134...",{18 Ronald Regan ...,18 Ronald Regan Hill,Shawnee Mission,Kansas,66225,3
200004,Goldi,Leaman,gleaman3@360.cn,Female,"[2027069459, 7042...",{7696 Calypso Jun...,7696 Calypso Junc...,Saint Paul,Minnesota,55166,5
200005,Brewster,Hallagan,bhallagan4@livejo...,Male,"[8134746319, 2152...","{942 Emmet Park, ...",942 Emmet Park,Albuquerque,New Mexico,87110,2
200006,Florence,Glashby,fglashby5@deviant...,Female,"[7571763565, 7134...",{664 Ridge Oak Ci...,664 Ridge Oak Circle,Virginia Beach,Virginia,23454,5
200007,Zollie,Philimore,zphilimore6@webed...,Male,"[7865450177, 8019...",{4 Continental Dr...,4 Continental Drive,Young America,Minnesota,55557,2
200008,Ferdinand,Ramelot,framelot7@alexa.com,Male,[8164055187],{44698 Arapahoe P...,44698 Arapahoe Park,Tucson,Arizona,85725,1
200009,Ruddy,Greystoke,rgreystoke8@digg.com,Male,null,"{null, null, null...",null,null,null,null,-1
200010,Gilles,Beeho,gbeeho9@webmd.com,Male,"[5599886694, 5201...",{9430 Reinke Park...,9430 Reinke Park,North Las Vegas,Nevada,89087,2


In [10]:
# 1st Use case - Count the total number of records across all the JSON files

spark.sql("select count(*) as total_records from user_sms_data")

total_records
1000000


In [11]:
# 2nd Use case - Find how many users are from the state New York

spark.sql("""select count(distinct(user_id)) as Total_users_from_NY
            from user_sms_data where 
            user_state = 'New York'""")

Total_users_from_NY
49576


In [12]:
# 3rd Use case - Find which state has the maximum number of post codes

spark.sql("""select user_state, count(distinct(user_postal_code)) as postal_code_counts
            from user_sms_data group by user_state order by postal_code_counts
            DESC LIMIT 1""")

user_state,postal_code_counts
California,206


In [13]:
# 4th Use case - Find the city with the maximum number of users (There are some nulls in user_city so we have to ensure to filter nulls out)

spark.sql("""select user_city, count(distinct(user_id)) as maximum_NoOfUsers
            from user_sms_data where user_city is not NULL 
            group by user_city order by maximum_NoOfUsers
            DESC LIMIT 1""")

user_city,maximum_NoOfUsers
Washington,28504


In [14]:
# 5th Use case - Count the total number of distinct users with a specific email

spark.sql("""select count(distinct(user_id)) from user_sms_data 
            where user_email like '%bizjournals.com'""")

count(DISTINCT user_id)
2015


In [15]:
# 6th Use case - Find the users with 4 phone numbers

spark.sql("""select count(distinct(user_id)) as unique_users from user_sms_data
            where countOf_PhoneNumbers = 4""")

unique_users
179041


In [16]:
# 7th Use case - Find the users which do not have any phone numbers linked to them

spark.sql("""select count(distinct(user_id)) as unique_users_without_pn
            from user_sms_data where countOf_PhoneNumbers = -1
            and user_phone_numbers is NULL""")

unique_users_without_pn
108981


In [18]:
# Now, let's write back the base df to HDFS home location in parquet format using the Spark writer API

user_sms_df.write.format("parquet").mode("overwrite").option("path", "/user/itv011181/Spark_Internals and Dataframes").save()

In [19]:
# Filtering out the gender count based on state-wise

spark.sql("""select user_state, 
                sum(total_MaleCounts) as Male,
                sum(total_FemaleCounts) as Female
                from
            (select user_state, case when user_gender = 'Male' then count(user_id) end as total_MaleCounts,
            case when user_gender = 'Female' then count(user_id) end as total_FemaleCounts
            from user_sms_data where user_state is not NULL
            and user_phone_numbers is not NULL
            group by user_state, user_gender)
            group by user_state
            order by user_state LIMIT 10""")

user_state,Male,Female
Alabama,9307,9178
Alaska,1882,1938
Arizona,9406,9543
Arkansas,2420,2416
California,49120,48716
Colorado,10128,10125
Connecticut,5797,5917
Delaware,1651,1654
District of Columbia,14212,14292
Florida,36692,36688


# Packaged the code into a .py file and ran it using the spark submit command with some defined configurations rather than using system default.

$ spark3-submit --master yarn --num-executors 2 --executor-cores 2 --executor-memory 4G --conf spark.dynamicAllocation.enabled=false spark_internals_submit.py
SPARK_MAJOR_VERSION is set to 3, using Spark3
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|user_gender|  user_phone_numbers|        user_address|
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+
| 200001|         Eirena|     Cutsforth|ecutsforth0@wisc.edu|     Female|[4197404036, 9173...|{8 Warrior Drive,...|
| 200002|          Marja|      Shopcott|mshopcott1@hexun.com|     Female|[9542037028, 2128...|{66 Prairieview T...|
| 200003|           Dawn|       Tointon|  dtointon2@ucsd.edu|     Female|[9523035647, 2134...|{18 Ronald Regan ...|
| 200004|          Goldi|        Leaman|     gleaman3@360.cn|     Female|[2027069459, 7042...|{7696 Calypso Jun...|
| 200005|       Brewster|      Hallagan|bhallagan4@livejo...|       Male|[8134746319, 2152...|{942 Emmet Park, ...|
| 200006|       Florence|       Glashby|fglashby5@deviant...|     Female|[7571763565, 7134...|{664 Ridge Oak Ci...|
| 200007|         Zollie|     Philimore|zphilimore6@webed...|       Male|[7865450177, 8019...|{4 Continental Dr...|
| 200008|      Ferdinand|       Ramelot| framelot7@alexa.com|       Male|        [8164055187]|{44698 Arapahoe P...|
| 200009|          Ruddy|     Greystoke|rgreystoke8@digg.com|       Male|                null|{null, null, null...|
| 200010|         Gilles|         Beeho|   gbeeho9@webmd.com|       Male|[5599886694, 5201...|{9430 Reinke Park...|
| 200011|           Axel|    Kitchinham|akitchinhama@nave...|       Male|        [2036913158]|{688 Granby Point...|
| 200012|         Moreen|      Pearmine|mpearmineb@github...|     Female|        [4047148129]|{3997 Russell Par...|
| 200013|       Joceline|       Adshead|jadsheadc@joomla.org|     Female|[3134837460, 3022...|{89 Fair Oaks Ter...|
| 200014|          Roxie|       Menichi|rmenichid@nationa...|     Female|[6148474898, 9034...|{3 Green Pass, Fo...|
| 200015|         Selena|         Flood|sfloode@delicious...|     Female|[7274706417, 4026...|{2 Butterfield Cr...|
| 200016|         Leoine|      Markovic|  lmarkovicf@usa.gov|     Female|                null|{null, null, null...|
| 200017|           Judi|      Svanetti|jsvanettig@theglo...|     Female|[2158389703, 8169...|{1 Sheridan Pass,...|
| 200018|     Hildegarde|         Hapke|     hhapkeh@nps.gov|     Female|        [8183692525]|{60 Oak Valley St...|
| 200019|          Hanni|       Dommett|hdommetti@tinyurl...|     Female|[7868201036, 7048...|{5 Vera Point, Wi...|
| 200020|         Rupert|       Gaskoin|rgaskoinj@mozilla...|       Male|[5623065887, 9198...|{52636 Heath Alle...|
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+
only showing top 20 rows

+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------------+----------+----------------+-------------+
|user_id|user_first_name|user_last_name|          user_email|user_gender|  user_phone_numbers|        user_address|         user_street|        user_city|user_state|user_postal_code|phone_numbers|
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------------+----------+----------------+-------------+
| 200001|         Eirena|     Cutsforth|ecutsforth0@wisc.edu|     Female|[4197404036, 9173...|{8 Warrior Drive,...|     8 Warrior Drive|           Dallas|     Texas|           75358|            4|
| 200002|          Marja|      Shopcott|mshopcott1@hexun.com|     Female|[9542037028, 2128...|{66 Prairieview T...|66 Prairieview Te...|           Joliet|  Illinois|           60435|            5|
| 200003|           Dawn|       Tointon|  dtointon2@ucsd.edu|     Female|[9523035647, 2134...|{18 Ronald Regan ...|18 Ronald Regan Hill|  Shawnee Mission|    Kansas|           66225|            3|
| 200004|          Goldi|        Leaman|     gleaman3@360.cn|     Female|[2027069459, 7042...|{7696 Calypso Jun...|7696 Calypso Junc...|       Saint Paul| Minnesota|           55166|            5|
| 200005|       Brewster|      Hallagan|bhallagan4@livejo...|       Male|[8134746319, 2152...|{942 Emmet Park, ...|      942 Emmet Park|      Albuquerque|New Mexico|           87110|            2|
| 200006|       Florence|       Glashby|fglashby5@deviant...|     Female|[7571763565, 7134...|{664 Ridge Oak Ci...|664 Ridge Oak Circle|   Virginia Beach|  Virginia|           23454|            5|
| 200007|         Zollie|     Philimore|zphilimore6@webed...|       Male|[7865450177, 8019...|{4 Continental Dr...| 4 Continental Drive|    Young America| Minnesota|           55557|            2|
| 200008|      Ferdinand|       Ramelot| framelot7@alexa.com|       Male|        [8164055187]|{44698 Arapahoe P...| 44698 Arapahoe Park|           Tucson|   Arizona|           85725|            1|
| 200009|          Ruddy|     Greystoke|rgreystoke8@digg.com|       Male|                null|{null, null, null...|                null|             null|      null|            null|           -1|
| 200010|         Gilles|         Beeho|   gbeeho9@webmd.com|       Male|[5599886694, 5201...|{9430 Reinke Park...|    9430 Reinke Park|  North Las Vegas|    Nevada|           89087|            2|
| 200011|           Axel|    Kitchinham|akitchinhama@nave...|       Male|        [2036913158]|{688 Granby Point...|    688 Granby Point|           Warren|      Ohio|           44485|            1|
| 200012|         Moreen|      Pearmine|mpearmineb@github...|     Female|        [4047148129]|{3997 Russell Par...|3997 Russell Parkway|North Little Rock|  Arkansas|           72199|            1|
| 200013|       Joceline|       Adshead|jadsheadc@joomla.org|     Female|[3134837460, 3022...|{89 Fair Oaks Ter...|89 Fair Oaks Terrace|  Shawnee Mission|    Kansas|           66220|            4|
| 200014|          Roxie|       Menichi|rmenichid@nationa...|     Female|[6148474898, 9034...|{3 Green Pass, Fo...|        3 Green Pass|       Fort Wayne|   Indiana|           46814|            4|
| 200015|         Selena|         Flood|sfloode@delicious...|     Female|[7274706417, 4026...|{2 Butterfield Cr...|2 Butterfield Cro...|       Sacramento|California|           94286|            3|
| 200016|         Leoine|      Markovic|  lmarkovicf@usa.gov|     Female|                null|{null, null, null...|                null|             null|      null|            null|           -1|
| 200017|           Judi|      Svanetti|jsvanettig@theglo...|     Female|[2158389703, 8169...|{1 Sheridan Pass,...|     1 Sheridan Pass|            Miami|   Florida|           33147|            5|
| 200018|     Hildegarde|         Hapke|     hhapkeh@nps.gov|     Female|        [8183692525]|{60 Oak Valley St...|60 Oak Valley Street|         Missoula|   Montana|           59806|            1|
| 200019|          Hanni|       Dommett|hdommetti@tinyurl...|     Female|[7868201036, 7048...|{5 Vera Point, Wi...|        5 Vera Point|    Wichita Falls|     Texas|           76310|            4|
| 200020|         Rupert|       Gaskoin|rgaskoinj@mozilla...|       Male|[5623065887, 9198...|{52636 Heath Alle...|   52636 Heath Alley|      Kansas City|  Missouri|           64160|            3|
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------------+----------+----------------+-------------+
only showing top 20 rows

+------------+
|unique_users|
+------------+
|       49576|
+------------+

+----------+------------------+
|user_state|total_postal_codes|
+----------+------------------+
|California|               206|
+----------+------------------+

+----------+-----------+
| user_city|total_users|
+----------+-----------+
|Washington|      28504|
+----------+-----------+

+-----------+
|total_users|
+-----------+
|       2015|
+-----------+

+------------------+
|total_unique_users|
+------------------+
|            179041|
+------------------+

+------------------+
|total_unique_users|
+------------------+
|            108981|
+------------------+